In [1]:
# Connect to AWS - Save Final Product

aws_access_key_id = 'ACCESS_KEY'
aws_secret_access_key = 'SECRET_ID'
sc._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", aws_access_key_id)
sc._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", aws_secret_access_key)

In [2]:
from datetime import datetime, date, timedelta
import dateutil.relativedelta
import pandas

# Setting Current Date - Based on Run Date
now = (datetime.date(datetime.now()))

# Is Today's CSV File Available? 
# If not, "today" actually needs to be yesterday

def whenistoday():
  today = now.strftime("%m-%d-%Y")
  
  try:
    csv_path = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv'.format(today)
    # Path Test
    df = pandas.read_csv(csv_path, nrows = 5)
    return(today)
  except:
    today = (now - timedelta(days = 1)).strftime("%m-%d-%Y")
    return(today)

today = whenistoday()

if now != today:
  now = (now - timedelta(days = 1))

# Setting Other Relative Dates
yesterday = (now - timedelta(days = 1)).strftime("%m-%d-%Y")
twoday = (now - timedelta(days = 2)).strftime("%m-%d-%Y")
threeday = (now - timedelta(days = 3)).strftime("%m-%d-%Y")
fourday = (now - timedelta(days = 4)).strftime("%m-%d-%Y")
fiveday = (now - timedelta(days = 5)).strftime("%m-%d-%Y")
sixday = (now - timedelta(days = 6)).strftime("%m-%d-%Y")
oneweek = (now - timedelta(days = 7)).strftime("%m-%d-%Y")
twoweek = (now - timedelta(days = 14)).strftime("%m-%d-%Y")
threeweek = (now - timedelta(days = 21)).strftime("%m-%d-%Y")

# Setting Relative Date - One Month Ago
# onemonth = now - dateutil.relativedelta.relativedelta(months=1)
# onemonth = datetime.strptime(str(onemonth) , '%Y-%m-%d').strftime("%m-%d-%Y")

In [3]:
# Prepare Loop for Import

from pyspark import SparkFiles, SparkContext
from pyspark.sql import SQLContext
import pandas as pd
import s3fs
import boto3

timeframes = [today, yesterday, twoday, threeday, fourday, fiveday, sixday, oneweek, twoweek, threeweek]

# Clear Old Files
s3 = boto3.resource('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
bucket = s3.Bucket('erikatestbucket')
bucket.objects.filter(Prefix="COVID-19/").delete()

Out[5]: []

In [4]:
# Loop to import files from Johns Hopkins CSSEGIS GitHub

for time in timeframes:
  url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv'.format(time)
  
  # Read CSV via Pandas
  pandasdf = pandas.read_csv(url)
  tables = dict
  
  # Write CSV via Pandas
  
  bytes_to_write = pandasdf.to_csv(None, index = False).encode()
  fs = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key)
  with fs.open('s3://erikatestbucket/COVID-19/{}.csv'.format(time), 'wb') as f:
    f.write(bytes_to_write)

In [5]:
# UDF to Import Tables (Without using dictionary)
def importdf(time):
  df = spark.read.csv('s3://erikatestbucket/COVID-19/{}.csv'.format(time), header = True, inferSchema = True)
  return(df)

# Import Tables
dftoday = importdf(today)
dfyesterday = importdf(yesterday)
dftwoday = importdf(twoday)
dfthreeday = importdf(threeday)
dffourday = importdf(fourday)
dffiveday = importdf(fiveday)
dfsixday = importdf(sixday)
dfoneweek = importdf(oneweek)
dftwoweek = importdf(twoweek)
dfthreeweek = importdf(threeweek)

In [6]:
# Creating New df
df = dftoday.select('FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Confirmed', 'Deaths', 'Recovered', 'Active')\
.withColumnRenamed("FIPS", "ID")\
.withColumnRenamed('Confirmed', 'Confirmed0')\
.withColumnRenamed('Deaths', 'Deaths0')\
.withColumnRenamed('Recovered', 'Recovered0')\
.withColumnRenamed('Active', 'Active0')

In [7]:
# Prepare Other DFs for Join - Rename Fields
# Loop/UDF Later

def cleantable(df, num):
  df = df.select('FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Confirmed', 'Deaths', 'Recovered', 'Active')\
  .withColumnRenamed("FIPS", "ID")\
  .withColumnRenamed('Confirmed', 'Confirmed-{}'.format(num))\
  .withColumnRenamed('Deaths', 'Deaths-{}'.format(num))\
  .withColumnRenamed('Recovered', 'Recovered-{}'.format(num))\
  .withColumnRenamed('Active', 'Active-{}'.format(num))
  return df
  
dfyesterday = cleantable(dfyesterday, '1')
dftwoday = cleantable(dftwoday, '2')
dfthreeday= cleantable(dfthreeday, '3')
dffourday = cleantable(dffourday, '4')
dffiveday = cleantable(dffiveday, '5')
dfsixday = cleantable(dfsixday, '6')
dfoneweek = cleantable(dfoneweek, '7')
dftwoweek = cleantable(dftwoweek, '14')
dfthreeweek = cleantable(dfthreeweek, '21')

In [8]:
# Join Tables Together

tables = [dfyesterday, dftwoday, dfthreeday, dffourday, dffiveday, dfsixday, dfoneweek, dftwoweek, dfthreeweek]
col_list=["ID","Admin2","Province_State", "Country_Region"]

def join(df, table):
  df = df.join(table, col_list, "left_outer")
  return df

for t in tables:
  df = join(df, t)
  
# Check that row count remains the same
df.count() == dftoday.count()

Out[92]: True

In [9]:
# Make New Fields - Reduce Table

Out[6]: '04-19-2020'

In [10]:
# Aggregate By Country - Reduce Table Size